# 1. Calculate the Origin-Destination Matrix to model travel across the city grids

In [ ]:
!pip install shapely
!pip install geopandas
!pip install osmnx
!pip install contextily

In [ ]:
#Standard and specialised module imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import osmnx as ox
import contextily as ctx
from shapely.geometry import Point, LineString, Polygon, MultiPolygon, shape, box
import geopandas as gpd
from geopy.geocoders import Nominatim, GoogleV3
from shapely.ops import unary_union
import networkx as nx
import yaml

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Dissertation/')

In [ ]:
polygon_filename = 'London_grids_with_boroughs_population.shp' # @param ["London_grids_with_population.shp", "London_grids_with_boroughs_population.shp"]

In [ ]:
gdf_london_polygons_3857_populated = gpd.read_file(polygon_filename)
gdf_london_polygons_3857_populated

In [ ]:
west, south, east, north = gdf_london_polygons_3857_populated.unary_union.bounds

## 1.1. TFL Tube Data 2020

### 1.1.1. Read Input Datasets (Monday to Thursday, Friday, Saturday and Sunday)

In [ ]:
tfl_tube_flows_data_MTT = pd.read_excel('Important Data/NBT20MTT_Outputs.xlsx', sheet_name='Link_Loads', header=2)
tfl_tube_flows_data_MTT

tfl_tube_flows_data_FRI = pd.read_excel('Important Data/NBT20FRI_Outputs.xlsx', sheet_name='Link_Loads', header=2)
tfl_tube_flows_data_FRI

tfl_tube_flows_data_SAT = pd.read_excel('Important Data/NBT20SAT_Outputs.xlsx', sheet_name='Link_Loads', header=2)
tfl_tube_flows_data_SAT

tfl_tube_flows_data_SUN = pd.read_excel('Important Data/NBT20SUN_Outputs.xlsx', sheet_name='Link_Loads', header=2)
tfl_tube_flows_data_SUN

In [ ]:
print(tfl_tube_flows_data_MTT.columns.tolist())

In [ ]:
tfl_tube_flows_data_filtered = tfl_tube_flows_data_MTT[['Line', 'Order', 'From Station', 'To Station', 'Total']].copy()
tfl_tube_flows_data_filtered.loc[:,'From Station'] = tfl_tube_flows_data_filtered['From Station'].str.replace(r'\bLU\b', 'London Underground', regex=True)
tfl_tube_flows_data_filtered.loc[:,'To Station'] = tfl_tube_flows_data_filtered['To Station'].str.replace(r'\bLU\b', 'London Underground', regex=True)
tfl_tube_flows_data_filtered

### 1.1.2. Generate a dataframe containing the distinct stations in London

In [ ]:
from_station_list = tfl_tube_flows_data_filtered['From Station'].dropna().tolist()
to_station_list = tfl_tube_flows_data_filtered['To Station'].dropna().tolist()

unique_stations_list = list(set(to_station_list).union(set(from_station_list)))
len(unique_stations_list)

df_stations = pd.DataFrame({'Station Name': unique_stations_list})
df_stations

### 1.1.3. Figure out the coordinates of the London Tube Stations

#### Retrieve API Key for the Google Maps geocoding API from the YAML file.

In [ ]:
with open('geocoding-api-key.yaml', 'r') as f:
    config = yaml.safe_load(f)

api_key = config['password']

In [ ]:
# Initialize the Nominatim geocoder
#using GoogleMaps API
geolocator = GoogleV3(api_key=api_key)
geolocator.geocode("Waterloo London Underground").point

# Function to get the Point for each station name
def fn_get_point_gmaps(station_name):
    location = geolocator.geocode(station_name + " Station, London, UK").point
    if location:
        return Point(location.longitude, location.latitude)  # Note: Longitude comes first, then Latitude
    else:
        location = geolocator.geocode(station_name + " , London, UK").point
        if location:
            return Point(location.longitude, location.latitude)
        else:
            return None

# Apply the function to the 'StationName' column and create a new 'Point' column
df_stations['Point'] = df_stations['Station Name'].apply(fn_get_point_gmaps)
df_stations

In [ ]:
df_stations = df_stations.rename(columns={'Point':'geometry'})
gdf_stations = gpd.GeoDataFrame(df_stations, crs='epsg:4326')
gdf_stations.crs

In [ ]:
gdf_stations = gdf_stations.to_crs('3857')
gdf_stations.count()

In [ ]:
#add the rest of the map in the background
fig, ax = plt.subplots(figsize=(30, 30))
gdf_stations.plot(ax=ax ,figsize=(30,30), alpha=0.5, edgecolor='r', linewidth=10, cmap='magma') #column = 'geometry'
gdf_london_polygons_3857_populated.plot(figsize=(30, 30), ax=ax ,alpha=0.5, edgecolor='k',linewidth=3)
for idx, row in gdf_london_polygons_3857_populated.iterrows():
    if 'NAME' in gdf_london_polygons_3857_populated.columns:
      label = row['NAME']  # Replace 'grid_index' with the desired column containing labels or information
    else:
      label = row["grid_index"]
    centroid_coords = row['geometry'].centroid.coords[0]
    ax.annotate(text=label, xy=centroid_coords, horizontalalignment='center', size=10)

ctx.add_basemap(ax, zoom=13)
#ax.set_xlim(west, east)
#ax.set_ylim(south, north)
plt.show()

In [ ]:
def fn_buildGriddedTubeDataset(gdf_tube_dataset_input, gdf_stations_london, gdf_population_data):

    gdf_tube_dataset_filtered_input = gdf_tube_dataset_input[['Line', 'Order', 'From Station', 'To Station', 'Total']].copy()
    gdf_tube_dataset_filtered_input.loc[:,'From Station'] = gdf_tube_dataset_filtered_input['From Station'].str.replace(r'\bLU\b', 'London Underground', regex=True)
    gdf_tube_dataset_filtered_input.loc[:,'To Station'] = gdf_tube_dataset_filtered_input['To Station'].str.replace(r'\bLU\b', 'London Underground', regex=True)
    print(gdf_tube_dataset_filtered_input)

    gdf_tube_dataset_filtered_input_coords = gdf_tube_dataset_filtered_input.merge(gdf_stations_london, left_on='From Station', right_on='Station Name', how='inner')
    gdf_tube_dataset_filtered_input_coords['From Station Coords'] = gdf_tube_dataset_filtered_input_coords['geometry']
    gdf_tube_dataset_filtered_input_coords = gdf_tube_dataset_filtered_input_coords.drop(['geometry', 'Station Name'], axis=1)

    gdf_tube_dataset_filtered_input_coords = gdf_tube_dataset_filtered_input_coords.merge(gdf_stations_london, left_on='To Station', right_on='Station Name', how='inner')
    gdf_tube_dataset_filtered_input_coords = gdf_tube_dataset_filtered_input_coords.rename(columns={'geometry': 'To Station Coords'})
    gdf_tube_dataset_filtered_input_coords = gdf_tube_dataset_filtered_input_coords.drop(['Station Name'], axis=1)
    print(gdf_tube_dataset_filtered_input_coords)

    gdf_tube_dataset_filtered_input_coords_from = gdf_tube_dataset_filtered_input_coords.copy()
    gdf_tube_dataset_filtered_input_coords_from['geometry'] = gdf_tube_dataset_filtered_input_coords_from['From Station Coords']
    gdf_tube_dataset_filtered_input_coords_from = gpd.GeoDataFrame(gdf_tube_dataset_filtered_input_coords_from, crs='3857')
    gdf_tube_dataset_filtered_input_coords_from


    start_join = gpd.sjoin(gdf_tube_dataset_filtered_input_coords_from.to_crs('3857'), gdf_population_data[['geometry', 'grid_index']].to_crs('3857'), how='left', predicate='within')
    start_join['From Grid ID'] = start_join['grid_index']
    start_join = start_join.drop(['index_right', 'grid_index'], axis=1)

    start_join['geometry'] = start_join['To Station Coords']
    end_join = gpd.sjoin(start_join.to_crs('3857'), gdf_population_data[['geometry', 'grid_index']].to_crs('3857'), how='left', predicate='within')
    end_join['To Grid ID'] = end_join['grid_index']
    end_join = end_join.drop(['index_right','grid_index'], axis=1)
    print(end_join)

    #Filter out stations that lie outside the London city limits
    end_join_filtered = end_join[~end_join['To Grid ID'].isnull() & ~end_join['From Grid ID'].isnull()]
    end_join_filtered

    return end_join_filtered

### 1.1.3. Calculate the Flow Data for MTT, FRI, SAT and SUN

In [ ]:
gdf_tube_flow_MTT = fn_buildGriddedTubeDataset(tfl_tube_flows_data_MTT, gdf_stations, gdf_london_polygons_3857_populated)
gdf_tube_flow_MTT

gdf_tube_flow_FRI = fn_buildGriddedTubeDataset(tfl_tube_flows_data_FRI, gdf_stations, gdf_london_polygons_3857_populated)
gdf_tube_flow_FRI

gdf_tube_flow_SAT = fn_buildGriddedTubeDataset(tfl_tube_flows_data_SAT, gdf_stations, gdf_london_polygons_3857_populated)
gdf_tube_flow_SAT

gdf_tube_flow_SUN = fn_buildGriddedTubeDataset(tfl_tube_flows_data_SUN, gdf_stations, gdf_london_polygons_3857_populated)
gdf_tube_flow_SUN

In [ ]:
#add the rest of the map in the background
fig, ax = plt.subplots(figsize=(30, 30))
gdf_tube_flow_MTT.plot(figsize=(30, 30), ax=ax, alpha=0.5, edgecolor='r', linewidth=30) #, column='geometry'
gdf_london_polygons_3857_populated.plot(figsize=(30, 30),alpha=0.5, ax=ax, edgecolor='k',linewidth=3) #'geometry'

for idx, row in gdf_london_polygons_3857_populated.iterrows():
    if 'NAME' in gdf_london_polygons_3857_populated.columns:
      label = row['NAME']  # Replace 'grid_index' with the desired column containing labels or information
    else:
      label = row["grid_index"]
    centroid_coords = row['geometry'].centroid.coords[0]
    ax.annotate(text=label, xy=centroid_coords, horizontalalignment='center', size=30)

ctx.add_basemap(ax, zoom=13)
ax.set_xlim(west, east)
ax.set_ylim(south, north)
plt.show()

## 1.2. Build the Origin-Destination Matrix

In [ ]:
def fn_generateODMatrix(gdf_flow_dataset, gdf_population_data):
    # Get the unique Grid IDs
    grid_ids = gdf_population_data['grid_index'].unique()

    # Create a weighted graph using NetworkX
    G = nx.Graph()

    # Add edges with total people as edge weights
    for index, row in gdf_flow_dataset.iterrows():
        from_grid_id = row['From Grid ID']
        to_grid_id = row['To Grid ID']
        total_people = row['Total']

        if G.has_edge(from_grid_id, to_grid_id):
            G[from_grid_id][to_grid_id]['weight'] += total_people
        else:
            G.add_edge(from_grid_id, to_grid_id, weight=total_people)

    # Use Floyd-Warshall algorithm to find shortest paths and total flow between all pairs of grids
    all_pairs_shortest_paths = dict(nx.floyd_warshall(G, weight='weight'))

    # Create a square matrix to store the OD flow
    od_matrix = pd.DataFrame(np.zeros((len(grid_ids), len(grid_ids))), index=grid_ids, columns=grid_ids, dtype=float)

    # Populate the OD matrix with total flow between all pairs of grids
    for from_grid_id, to_grid_data in all_pairs_shortest_paths.items():
        for to_grid_id, total_flow in to_grid_data.items():
            od_matrix.at[from_grid_id, to_grid_id] = total_flow

    # Normalize the OD matrix to values between 0 and 1
    #od_matrix_normalized = (od_matrix - od_matrix.min().min()) / (od_matrix.max().max() - od_matrix.min().min())

    # Normalize the OD matrix to values between 0 and 1
    row_sums = od_matrix.sum(axis=1)
    od_matrix_normalized = od_matrix.div(row_sums, axis=0)
    #od_matrix_normalized = od_matrix.fillna(0.0)
    #od_matrix_normalized = round(od_matrix_normalized,5)

    for index, row in od_matrix_normalized.iterrows():
        if not np.isclose(row.sum(), 1.0):  # Handle potential floating-point inaccuracies
            od_matrix_normalized.loc[index] = row / row.sum()

    # Handle division by zero and set NaN values to 0
    od_matrix_normalized = od_matrix_normalized.fillna(0.0)

    # Optionally, you can add human-readable labels for rows and columns
    od_matrix_normalized_labeled = pd.DataFrame(od_matrix_normalized.values, index=grid_ids, columns=grid_ids)

    od_matrix_normalized_labeled_numpy =  od_matrix_normalized_labeled.to_numpy()

    return od_matrix_normalized_labeled_numpy

### 1.2.1. Generate OD matrix for MTT, FRI, SAT & SUN

In [ ]:
od_matrix_normalized_labeled_MTT = fn_generateODMatrix(gdf_tube_flow_MTT, gdf_london_polygons_3857_populated)
od_matrix_normalized_labeled_MTT

od_matrix_normalized_labeled_FRI = fn_generateODMatrix(gdf_tube_flow_FRI, gdf_london_polygons_3857_populated)
od_matrix_normalized_labeled_FRI

od_matrix_normalized_labeled_SAT = fn_generateODMatrix(gdf_tube_flow_SAT, gdf_london_polygons_3857_populated)
od_matrix_normalized_labeled_SAT

od_matrix_normalized_labeled_SUN = fn_generateODMatrix(gdf_tube_flow_SUN, gdf_london_polygons_3857_populated)
od_matrix_normalized_labeled_SUN

In [ ]:
#Generate the final OD matrix for a whole week (7 days starting from Monday till Sunday)
od_matrix_normalized_weekly = np.stack((od_matrix_normalized_labeled_MTT, od_matrix_normalized_labeled_MTT, od_matrix_normalized_labeled_MTT,
                                        od_matrix_normalized_labeled_MTT, od_matrix_normalized_labeled_FRI,
                                        od_matrix_normalized_labeled_SAT, od_matrix_normalized_labeled_SUN))

In [ ]:
od_matrix_normalized_weekly.shape

In [ ]:
od_matrix_normalized_weekly[0].sum(axis=1)